# VacationPy

## Imports

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

## Storing results as DataFrame

In [10]:
cities_df = pd.read_csv("../WeatherPy/city_weather_data.csv")
cities_df.head()

,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date
0,-14.936039,13.322562,lubango,ao,69.80,56.0,0.0,3.44,1610528669
1,39.658409,-48.245995,torbay,ca,28.99,93.0,75.0,8.05,1610528850
2,-89.979413,37.550464,port elizabeth,za,71.60,83.0,90.0,13.80,1610528788
3,-35.580137,165.168906,ahipara,nz,65.88,84.0,10.0,6.15,1610528850
4,-63.583940,-38.583740,ushuaia,ar,41.00,75.0,0.0,8.05,1610528649
...,...,...,...,...,...,...,...,...,...
500,50.440424,-110.868744,medicine hat,ca,45.00,65.0,1.0,14.97,1610528977
501,8.361826,124.960650,maluko,ph,80.01,85.0,100.0,5.01,1610528978
502,56.959632,1.215939,boddam,gb,35.60,80.0,96.0,10.36,1610528978
503,16.191234,-76.291348,morant bay,jm,75.20,78.0,20.0,8.05,1610528978


## Humidity Heatmap

In [11]:
# Create a heat map that displays the humidity for every city from the part I of the homework.

# Configuring gmaps with API key
gmaps.configure(api_key=g_key)

# Storing Latitude and Longitude values into  locations 
locations = cities_df[["Latitude", "Longitude"]]

# Storing Humidity as the weight
humidity = cities_df["Humidity"]

In [36]:
# Create the map and a humidity Heatmap layer
fig = gmaps.figure(center=(20, 0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Specific Weather Criteria

In [39]:
cities_df.dtypes


Latitude               float64
Longitude              float64
City                    object
Country                 object
Max Temperature (F)    float64
Humidity               float64
Cloudiness             float64
Wind Speed (mph)       float64
Date                     int64
dtype: object

In [43]:
# Narrowing down the DataFrame to find ideal weather conditions: 

ideal_weather = cities_df.loc[((cities_df["Max Temperature (F)"] >= 75) & 
                               (cities_df["Max Temperature (F)"] <= 87) & 
                               (cities_df["Wind Speed (mph)"] <= 10) & 
                               (cities_df["Cloudiness"] < 5) & 
                               (cities_df["Humidity"] < 50)), :]  

ideal_weather

,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date
5,-56.473162,104.629318,busselton,au,81.00,36.0,0.0,1.01,1610528771
61,-27.198298,147.966471,roma,au,84.20,31.0,0.0,9.31,1610528865
84,9.507956,-8.297668,odienne,ci,77.77,18.0,0.0,4.76,1610528871
207,24.588792,40.878010,mecca,sa,81.82,20.0,0.0,2.21,1610528690
244,10.532979,-7.947414,mandiana,gn,76.28,19.0,0.0,6.60,1610528913
369,23.696393,74.587540,banswara,in,81.46,16.0,0.0,5.79,1610528945
449,17.212518,103.140275,nong han,th,78.01,31.0,0.0,3.44,1610528964


## Hotel Map

In [48]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
# Store into variable named hotel_df.

hotel_df = ideal_weather.copy()

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

hotel_df

,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date,Hotel Name
5,-56.473162,104.629318,busselton,au,81.00,36.0,0.0,1.01,1610528771,
61,-27.198298,147.966471,roma,au,84.20,31.0,0.0,9.31,1610528865,
84,9.507956,-8.297668,odienne,ci,77.77,18.0,0.0,4.76,1610528871,
207,24.588792,40.878010,mecca,sa,81.82,20.0,0.0,2.21,1610528690,
244,10.532979,-7.947414,mandiana,gn,76.28,19.0,0.0,6.60,1610528913,
369,23.696393,74.587540,banswara,in,81.46,16.0,0.0,5.79,1610528945,
449,17.212518,103.140275,nong han,th,78.01,31.0,0.0,3.44,1610528964,


In [88]:
# # Storing Latitude and Longitude values into  locations 
#locations = cities_df[["Latitude", "Longitude"]]
  #lat = hotel_df['Latitude'][x]
    #long = hotel_df['Longitude'][x]
    #city = citipy.nearest_city(lat, long

# setting empty hotel name list
hotels = []                               

# itterating through each row using the index
for x in hotel_df.index:
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # geocoordinates
    lat = hotel_df['Latitude'][x]
    long = hotel_df['Longitude'][x]
    locations = lat, long
    target_search = "Hotel"
    target_radius = 5000
    target_type = "lodging"
                                
    # setting up a parameters dictionary
    params = {"location": locations, 
              "keyword": target_search, 
              "radius": target_radius, 
              "type": target_type, 
              "key": g_key}                            
     
        
    try:    
    # running a request using our params dictionary
        response = requests.get(base_url, params=params)

    # convert response to json
        hotel_data = response.json()    
     
        hotel = hotel_data["results"][0]["name"]
    
    # appending the hotel to the hotel list    
        hotels.append(hotel)    
    
    except IndexError:
        hotels.append(np.nan)
    
    
#adding list of hotels to data frame
hotel_df["Hotel Name"] = hotels
hotel_df

,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date,Hotel Name
5,-56.473162,104.629318,busselton,au,81.00,36.0,0.0,1.01,1610528771,NaN
61,-27.198298,147.966471,roma,au,84.20,31.0,0.0,9.31,1610528865,NaN
84,9.507956,-8.297668,odienne,ci,77.77,18.0,0.0,4.76,1610528871,NaN
207,24.588792,40.878010,mecca,sa,81.82,20.0,0.0,2.21,1610528690,NaN
244,10.532979,-7.947414,mandiana,gn,76.28,19.0,0.0,6.60,1610528913,NaN
369,23.696393,74.587540,banswara,in,81.46,16.0,0.0,5.79,1610528945,NaN
449,17.212518,103.140275,nong han,th,78.01,31.0,0.0,3.44,1610528964,NaN


In [98]:

# setting empty hotel name list
hotels = []

#base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():

    # setting variables 
    city = row['City']
    lat = row['Latitude']
    long = row['Longitude']
    locations = f"{lat},{lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"
                              
    # setting up a parameters dictionary
    params = {"location": locations, 
              "keyword": target_search, 
              "radius": target_radius, 
              "type": target_type, 
              "key": g_key}                            
      
    print("--------------------------------------")  
    print(f"Retrieving Results for Index: {city}.")
    
    # adding to data frame using try and except incase no data    
    try:    
    # running a request using our params dictionary
        response = requests.get(base_url, params=params)

    # convert response to json
        hotel_data = response.json()    
     
    # appending the hotel to the hotel list    
        hotels.append(hotel_data["results"][0]["name"])    
    
        print("Results Found")
    
    except IndexError:
        print("Unable to retrieve results... skipping.")
        hotels.append(np.nan)    
    
#adding list of hotels to data frame
hotel_df["Hotel Name"] = hotels
hotel_df



Retrieving Results for Index: busselton.
Unable to retrieve results... skipping.
Retrieving Results for Index: roma.
Unable to retrieve results... skipping.
Retrieving Results for Index: odienne.
Unable to retrieve results... skipping.
Retrieving Results for Index: mecca.
Unable to retrieve results... skipping.
Retrieving Results for Index: mandiana.
Unable to retrieve results... skipping.
Retrieving Results for Index: banswara.
Unable to retrieve results... skipping.
Retrieving Results for Index: nong han.


,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date,Hotel Name
5,-56.473162,104.629318,busselton,au,81.00,36.0,0.0,1.01,1610528771,NaN
61,-27.198298,147.966471,roma,au,84.20,31.0,0.0,9.31,1610528865,NaN
84,9.507956,-8.297668,odienne,ci,77.77,18.0,0.0,4.76,1610528871,NaN
207,24.588792,40.878010,mecca,sa,81.82,20.0,0.0,2.21,1610528690,NaN
244,10.532979,-7.947414,mandiana,gn,76.28,19.0,0.0,6.60,1610528913,NaN
369,23.696393,74.587540,banswara,in,81.46,16.0,0.0,5.79,1610528945,NaN
449,17.212518,103.140275,nong han,th,78.01,31.0,0.0,3.44,1610528964,James & oi homestay


In [85]:
locations


(-56.47316195201925, 104.6293176922556)

In [86]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

import json


# geocoordinates
target_coordinates = "43.6187102, -116.2146068"
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}


# google places api needs info 
# so that is what is above
# we have put it in a dictionary
# passing these parameters into the response using params
# this is in our call to requests.get
# so like how we put the parameters in as query string parameters before 
# this is a different way to do it
# this way is a lot neater and managable 
# the values we put as the keys above would be values seen in the google documentation.
# they are the parameters that are added to the url along with the values they are referencing

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))



{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 43.6139805,
                    "lng": -116.2037792
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.61524747989272,
                        "lng": -116.2022788701073
                    },
                    "southwest": {
                        "lat": 43.61254782010728,
                        "lng": -116.2049785298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "The Grove Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 553,
                    "html_attributions": [
                        "<a 

In [63]:
print(places_data["results"][0]["name"])

The Grove Hotel


Retrieving Results for Index 5: busselton.
Missing field/result... skipping.
------------
Retrieving Results for Index 61: roma.
Missing field/result... skipping.
------------
Retrieving Results for Index 84: odienne.
Missing field/result... skipping.
------------
Retrieving Results for Index 207: mecca.
Missing field/result... skipping.
------------
Retrieving Results for Index 244: mandiana.
Missing field/result... skipping.
------------
Retrieving Results for Index 369: banswara.
Missing field/result... skipping.
------------
Retrieving Results for Index 449: nong han.
Closest hotel in nong han is James & oi homestay.
------------
-------End of Search-------
